
 # 主题模型
   从生成文档的角度来产生文档的主题，应用了大量的概率计算以及参数估计的方法来寻找每一篇文档的主题分布以及每个主题的词语分布情况。
  

1. LDA模型中一篇文档的生成

    在引入LDA模型的文档的生成过程前，先讲解一下PLSA模型的文档生成过程。
    一篇文档可以有很多的主题，每个主题包含很多的词语，文档的生成我们可以看成是在摇骰子，先找到一个主题，然后再从该主题下摇到一个词语，反复这样摇N次，则可以形成一篇有N个字的文档。在此过程中文档的主题分布和每个主题的词语分布是事先给定的。
    
    LDA模型和PLSA模型不一样的是，LDA模型的“两个”分布是随机产生的。其随机分布均服从于迪利克雷分布。

    LDA模型中（一篇）文档的生成过程为：
    
      step1： 在迪利克雷分布alpha中随机产生主题模型p1

      step2： 从主题分布中取样生成文档i中第j个单词的主题Z_ij  (生成一个主题)

      step3： 从狄利克雷分布beta中取样生成主题z_ij对应的词语分布

      step4： 利用生成的词语分布生成词汇w_ij

    LDA模型可以写成两个步骤：

        alpha ----> seta_m -----> zmn

        beta -----> empslo -----> wmn


 ![LDA模型的过程](https://img-blog.csdn.net/20141117152903751)   

 ![LDA实例过程](https://img-blog.csdn.net/20141127192035125)    
 
  整个过程中可以看成：
    
    先验分布  + 产生主题/词语  = 后验分布 

    先验分布 + 多项式分布   = 后验分布 

    Dirichlet + multinational = Dirichlet

    所以整个LDA过程可以看作是dirichlet-multinational共轭结构，这样的结构在整个过程中总共有M+K个。

  ![Dirichlet-multinational](https://img-blog.csdn.net/20141120174743000)
  
  ![D-M过程](https://img-blog.csdn.net/20141120174912796)

    整个过程中需要求解（均是条件概率）： 
      每个文档中主题的概率
      整个语料库中topic的生成的概率
      每个文档中词语出现的概率
      整个文档中词生成的概率

     有了上面的这些概率可以求得 已知两个先验分布的条件下主题和词语的联合分布


  而现实中我们首先知道的是词语的情况w，现在我们的目标是求解p(z|w)

  从上面的过程中我们已知联合分布 以及 条件分布 ---从这一点上看有点向Gibbs采样的过程。而我们现在的目标是利用词来采样主题，使得满足联合分布这一平稳分布

2. LDA模型的具体步骤

    主题分布和词语分布的先验分布选为迪利克雷分布的原因：我们在上述生成一个文档的时候采取的方法是摇色子的方法，在每一次摇色子的过程中，每个主题按照一定的概率出现，这个过程类似于多项式分布，而迪利克雷分布是多项式分布的共轭分布因此，采用迪利克雷分布作为先验分布。

    
  在实际操作中我们的文档是事先给定的，随机不需要生成该文档，但是我们可以认为我们的文档是利用上述操作形成的。LDA推断主题是一个反推的过程。

  在反推的过程中我们知道的量有：p(di) 、 p(wj|di) 

  上面2个主要的条件概率的求解：

   1. 整个语料库中topic生成的概率
     
     alpha ----主题分布的先验分布

     Zm  ----表示第m篇文档的主题情况。

     seta  ----先验分布产生的主题分布

   ![zm的概率](https://img-blog.csdn.net/20141121100156011) 

   2. 整个语料库中词产生的概率

     w=(w1,w2,.....wn)  整个语料库总共有n个单词（最开始要给所有的主题和词语进行编号）

     z=(z1,z2,...zk)   整个语料库中总共有k个主题

     而由于整个生成文档的过程是先摇doc-topic骰子，然后再摇topic-word骰子，而且生成的单词之间是独立且可交换的。假设整个语料库中总共有N个单词，因此总共摇了2N次骰子。由于主题的独立性和词语的独立性，我们可以将生成文档的过程进行改进：

     step1:前N次先生成N个主题

     step2:按照这些主题逐一的产生单词

     在这种情况下：

        w'=(w(1),w(2),...w(k))

        z'=(z(1),z(2),...z(k))

        w(k)表示第k个主题产生的词语

        z(k)表示词语的主题是K所对应的主题编号，其中z(k)的分量均为k

    ![整个文档生成词语的概率](https://img-blog.csdn.net/20141121100131691)

  利用这两个概率可以求得：

    ![联合分布](https://img-blog.csdn.net/20141121100200504)

  
3. LDA与Gibbs

  现实中我们是利用很多的文档来获取主题，在上面LDA主题模型中，在上面文档生成过程中，其主题分布以及每个主题分布的产生都是通过先验分布采样得来的。在上面的解释中，我们已经知道其主题分布以及词分布的先验分布为dirichlet分布。所以我们在进行一个文档的生成过程中需要不断的采样主题以及采样词语，现在我们所要做的是在已知词语的情况下去寻找每个其主题，这样的话就将整个文档生成的过程完整了。
  
  已知词语w，从上面的文档的生成过程中求得联合分布p(w,z|seta,empslo)，根据Gibbs算法的要求，我们在采样满足一个分布的样本的时候，在多维情况下，我们的状态转移概率则为在除去轴的变量的条件下求i值得概率，只有找到这个概率我们才能采样得到满足（w,z）的样本。 因此需要计算状态转移概率矩阵
  
        p(zi=k|Z~i,w)。
  
  语料库Z中的第i个词所对应的topic为Zi i={m,n} 表示第m篇文档的第n个词。用~i表示除去i所代表的词。
  
  其中Z~i表示语料库中除去当前i的情况的剩余的主题向量，W~i表示除去当前i情况的剩余的剩余的词向量。我们假设第i个所对应的词为t，则有：

       p(zi=k,wi=t|z~i,w~i)

  LDA主题模型可以看作是一个词袋模型，其语料库中主题的采样以及词语的采样都是独立的，因此在整个模型中减少这个词以及这个主题并不会影响原来模型的Dirichlet-multional共轭结构（zi=k,wi=t 只涉及到第m篇文档以及第k个topic，因此在上面的概率的计算中也只会涉及两个共轭结构，即生成第m篇文档和第k个topic。），只会使得这种共轭结构的个数减少，因此主题分布以及词分布的后验分布仍然是Dirichlet分布---只是分布的参数个数发生了变化。

        主题分布：
          p(seat_m|z~i,w~i) 服从 Dir(seat_m|Nm~i+alpha)

        词分布： p(empsol_k|z~i,w~i) 服从 Dir(empsol_k|Nk~i+beat)

        其中Nm~i 表示第m篇文档中除去第i个下标所对应的主题之后每个主题产生的词的个数
        其中Nk~i 表示第k个主题产生的词中除去第i个下标所对应之后每个词出现的个数
  
  因此状态转移概率矩阵的计算为：

   ![状态转移概率](https://img-blog.csdn.net/20141121135231473)


  最后参数的估计应用到了dirichlet分布的性质。





4. LDA模型的训练

   整个LDA主题模型我们需要求解的参数有：

       M个主题分布 以及K个词分布
   
   LDA最终的目标：是对于一篇新的文档来说，能够计算这篇文档的主题分布

   LDA训练过程：

      *   随机初始化，对语料中的每个词w随机的赋予一个主题编号；
      *   重新扫描语料库，对每个词w，按照Gibbs sample 公式重新采样topic
      *   重复采样，知道Gibbs过程收敛
      *   统计语料库中的topic-word出现的频率矩阵，该矩阵则为LDA模型
   
   具体在实践的时候我们需要要做的步骤：

      *   对文档进行处理--停用词，切分，形成词典
      *   模型的训练
      *   结果的展示  








 # **Gibbs**

  使用采样的方法主要是为了产生服从某个分布的样本点。常见的分布fenuniform、beta、gauss、normal、指数分布等这些分布的伪随机数我们可以利用python中的random模块的如betavariate等这些函数来产生，而有的时候我们面对的分布可能很是未知或者说可能没有很明显的规律，因此这个时候需要用到Gibbs采样方法，这样产生的一系列的样本是严格服从我们的平稳分布。

  Gibbs方法是MCMC方法的拓展。

---

 # 1.MCMC

  已知一个分布现在随机生成服从该分布的任意一个样本

  此时有一个很重要的定理---在一个非周期的马尔科夫链中，且其任意两个状态之间是联通的，那么pij^n存在且与i无关系
，最后的N阶转移概率矩阵每一行都收敛于pei向量

     lim Pij^n=pei(j)

     pei=(pei(1),pei(2),....pei(j),...pei(N))  总共有N个状态

对于一个序列x0,其服从分布pei(0),其x0共有m个状态，其状态转移概率矩阵为p

经过一次转移之后，我们的x1~pei_1=pei_n*p

经过两次转移之后，我们的x2~pei_2=pei_1*p=pei_0*P^2

而由上述的定理我们知道随着n-->inf pij^n=pei(j)

即经过n次转移之后，状态转移矩阵趋于收敛，也即：

     xn~pei_n-1 * p=pei
     x(n+1)~pei_n * P= pei
     ...
     
     因此此时我们可以发现最后得到的xn,x(n+1),...都是平稳分布pei的样本
     
     因此这样就找到了一系列服从于同一个分布的样本集合
     
     ---可以利用上面的理论进行样本的生成。
     
现在我们的难点是，我们如何构造一个状态转移概率矩阵使得最后收敛的分布是我们想要求得分布。
---此时引入了一个定理细致平稳条件：

   若对于任意一个非周期得马尔科夫链满足：

           pei(i)*pij=pei(j)*pji  ---->有了这个条件我们可以推出来pei*P=pei的唯一解就是pei
           则pei是平稳分布
   
   在初次转移的时候 pei(i)=pei0(i)=p(i),pei(j)=pei0(j)=p(j)
   
       初次转移之前：p(i) * pij =p(j) * pji
   
   一般来说这个等式是不成立的。为了让这个等式成立两边引入变量：
   
       p(i) * pij * alpha(i,j) = p(j) * pji * alpha(j,i)
   
   此时满足：alpha(i,j)=p(j) * pji ; alpha(j,i) = p(i) * pij 这样上面这个等式则成立，alpha可以看成是
   
   在状态进行转移的时候的接受概率。
       
       则上面的式子则变成 p(i) * qij = p(j) * qji。 
       
       则马尔科夫链Q的平稳分布为p
       
       初始马尔科夫链P的转移矩阵为 Pij ---->新的马尔科夫链Q的转移矩阵为 Pij*alpha(i,j)
       
MCMC算法：

    step1：初始化马尔科夫链的状态X0=pei0
    step2：t=0,1,2,....，不断地循环下面的过程：
        a: 第t个时刻马尔科夫链的的状态为：Xt=xt,则采样y~q(y|xt)=P(xt,y)--状态转移概率 
        b: 从均匀分布采样u ~ uniform[0,1]
        c: 如果u < alpha(xt,y)=p(y)*p(xt|y)=p(y)*P(y,xt) （接受概率比此时随机产生的概率U还大的话，
           则进行转移）则x_t+1 = y
        d：否则不接受的话则  x_t+1 = xt
        在上面的整个过程中我们还需要一个数组用来记录整个过程最后的状态转移矩阵，观察其是否最后收敛到我们
        的平稳分布。


算法的改进：

    有的时候我们计算得到的alpha 会比较小，这样要生成服从平稳分布p的序列的速度很慢，为了增大效率同时又不破
    坏细致平稳条件，则
        alpha(i,j) = min (p(j)*pji / p(i)*pij, 1)                   
        
此时还有两个需要注意的地方 y的产生以及alpha的计算
产生Y  已知某个概率分布的概率求解相应的状态值，也即要用到累计分布的反函数

---离散情况下：

    y的产生：
        1.以一定的转移概率进行求解P[y|xt]下所对应的状态
        2.上面的一定的概率一般可以认为，当前状态所在状态转移概率矩阵中最大概率所对应的状态即可
        
    相应的有了Y则alpha比较好计算了 
---连续情况下：

    由于所有的分布在一定的条件下都可以转换为正态分布，因此我们在进行MCMC产生序列的时候，我们的最终要求的
    平稳分布也可以转换称为求解相应正态分布情况下的平稳分布即可。
    此时y的产生和alpha的计算可以利用norm的rvs和pdf函数进行计算,可以直接调取scipy中的包即可求解
    from scipy.status import norm 
    y=norm.rvs()
    alpha的计算可以利用norm.pdf进行计算          




# 2.GIBBS采样

---
在上面的MCMC方法中，我们的接受概率是小于1的，有的时候如果计算出来的接受概率小于均匀分布随机数u
的话则会拒绝接受，因此这个时候我们t+1的状态和上面一个状态时一致的，此时这个过程采样达到平稳分布的效率不是很高
Gibbs算法则要求接受概率=1

对于一个二维平面的情况来说：

    1.点A(x1,y1), 点B(X1,Y2) 这两个点在一条垂直线上
    p(x1,y1) * p(y2|x1) = p(x1) * p(y1|x1) * p(y2|x1)
    p(x1,y2) * p(y1|x1) = p(x1) * p(y2|x1) * p(y1|x1)
    则从上面的情况看，上述两个等式相等：
        p(x1，y1) * p(y2|x1) = p(x1,y2) * p(y1|x1)
        p(A) * p(A--->B) = P(B) * P(B---->A)  此时xa=xb=x1
    2.点C(x2,y1)
        p(x1,y1) * p(x2|y1) = p(y1) * p(x1|y1) * p(x2|y1)
        p(x2,y1) * p(x1|y1) = p(y1) * p(x2|y1) * p(x1|y1)
    则从上面的情况看，上述两个等式相等：
        p(x1，y1) * p(x2|y1) = p(x2,y1) * p(x1|y1)
        p(A) * p(A--->C) = P(C) * P(C---->A)  此时Ya=Yc=Y1   
   
   结合上面的情况的状态转移概率为：

        Q(A--->B)=P(YB|X1)  此时A,B的横坐标相等，两者在一条垂直于x轴的直线上
        Q(A--->C)=P(XC|YA)  此时A,C的纵坐标相等，两者在一条平行x轴的直线上
        Q(A--->D)=0         此时A,D的坐标之间没有任何的关系
   
   所以在Gibbs中二维平面上的点进行转移的时候，只沿着x轴和y轴进行转移，因此GIBBs的状态转移图形是各种直线段组成的
   
   二维情况下的GIbbs算法：

       step1：初始化状态（x0,y0)
       step2:for t=0 到 转移次数：
               y_(t+1) ~ p(y|xt)   ---纵向转移                 
               x_(t+1) ~ p(x|y_(t+1))   ---横向转移
               此时转移得到两个点：
                   (xt,y_(t+1))  (x_(t+1),y_(t+1))
       
           


# Gibbs采样的实例

In [0]:
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import multivariate_normal
import random
import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 
#平稳分布
samplescore=multivariate_normal(mean=[5,-1],cov=[[1,1],[1,4]])

#产生随机数
def get_y(x,miu1,miu2,s1,s2,rho):
    return  (random.normalvariate(m2 + rho * s2 / s1 * (x - m1), math.sqrt((1 - rho ** 2) * (s2**2))))

def get_x(y,miu1,miu2,s1,s2,rho):
    return (random.normalvariate(m1 + rho * s1 / s2 * (y - m2), math.sqrt((1 - rho ** 2) * (s1**2))))

N=5000
m1=5
m2=-1
s1=1
s2=2
rho=0.5
x_res=[]
y_res=[]
point=[]
prob_matrix=[]
y=m2
for t in range(0,N):
    x=get_x(y,m1,m2,s1,s2,rho)
    y=get_y(x,m1,m2,s1,s2,rho)
    z=samplescore.pdf([x,y])
    x_res.append(x)
    y_res.append(y)
    point.append([x,y])
    prob_matrix.append(z)

#最后求解采样得到的样本的均值和方差，观察其是否和我们要采样的分布一致

point=np.array(point)
m=np.mean(point,axis=0)
var=np.cov(point.T)
print('采样得到的样本点的均值:')
print(m)
print('采样得到的样本点的协方差:')
print(var)

print('目标平稳分布的均值:')
print([m1,m2])
print('目标平稳分布的协方差')
print([[s1**2,rho*s1*s2],[rho*s1*s2,s2**2]])

fig = plt.figure()
ax = Axes3D(fig, rect=[0, 0, 1, 1], elev=30, azim=20)
ax.scatter(x_res, y_res, prob_matrix,marker='o')
plt.show()      

'''
num_bins = 50
plt.hist(x_res, num_bins, normed=1, facecolor='green', alpha=0.5)
plt.hist(y_res, num_bins, normed=1, facecolor='red', alpha=0.5)
plt.title('Histogram')
plt.show()
'''                            

# LDA主题模型使用gensim模块训练的实例

https://github.com/l11x0m7/LDA/blob/master/lda_realize.py  另外一个实例

nltk模块进行语料切分：

  https://www.cnblogs.com/zrmw/p/10875684.html

In [12]:
from nltk.tokenize import RegexpTokenizer
#语料切分模块
from nltk.corpus import stopwords
'''import nltk  有的时候需要先下载停用词
   nltk.download('stopwords')'''
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
#from nltk.tokenize import word_tokenize

import nltk
#nltk.download('stopwords') 
#tokenizer = RegexpTokenizer(r'\w+')
 
# create English stop words list
en_stop = set(stopwords.words('english'))
#word_tokens = word_tokenize(example_sent)

#filtered_sentence = [w for w in word_tokens if not w in stop_words] 
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
# create sample documents
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health." 
 
# compile sample documents into a list
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]
 
# list for tokenized documents in loop
texts = []
 
# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)  #进行语料的切分 
 
    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
print(dictionary.token2id)    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
 
# generate LDA model

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20) 
print('model1')
print(ldamodel.print_topics(num_topics=3, num_words=3))

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary, passes=20)
print('model2,每个主题的词分布')
print(ldamodel.print_topics(num_topics=2, num_words=4))

for i in range(len(doc_set)):
  print(doc_set[i],end=',')
  



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
{'brocolli': 0, 'brother': 1, 'eat': 2, 'good': 3, 'like': 4, 'mother': 5, 'around': 6, 'basebal': 7, 'drive': 8, 'lot': 9, 'practic': 10, 'spend': 11, 'time': 12, 'blood': 13, 'caus': 14, 'expert': 15, 'health': 16, 'increas': 17, 'may': 18, 'pressur': 19, 'suggest': 20, 'tension': 21, 'better': 22, 'feel': 23, 'never': 24, 'often': 25, 'perform': 26, 'school': 27, 'seem': 28, 'well': 29, 'profession': 30, 'say': 31}
model1
[(0, '0.059*"drive" + 0.059*"brother" + 0.059*"mother"'), (1, '0.096*"good" + 0.096*"brocolli" + 0.067*"eat"'), (2, '0.065*"drive" + 0.065*"health" + 0.064*"pressur"')]
model2,每个主题的词分布
[(0, '0.089*"brocolli" + 0.089*"good" + 0.071*"brother" + 0.071*"mother"'), (1, '0.081*"health" + 0.059*"drive" + 0.059*"pressur" + 0.035*"increas"')]


上面讲了平稳分布序列的产生，对于常见的分布的随机数是如何产生的此处也加以解释

 # uniform 分布

 

 # 其他常见分布
   
   其他常见的分布的随机数都可以利用uniform分布来产生

